In [ ]:
import os
os.chdir("..")

In [ ]:
import polars as pl
import missingno as msno
import polars.selectors as cs
from src.data.data_process import DataClean

dc = DataClean()

In [ ]:
dc.dataset()

In [ ]:
df = dc.pull_wb()
df = df.filter(pl.col("year") >= 1990)
data = pl.read_csv("data/external/countries.csv")
df = df.join(data, on="country", how="inner", validate="m:1")
df = df.select(
    pl.col(
        'year',
        'country',
        'fertility_rate',
        'gdp_per_capita',
        'government_expenditure',
        'trade',
        'life_expectancy',
        'gdp_per_capita_growth',
        'inflation',
        'population_growth',
        'gross_capital_formation',
        'domestic_bank',
        'advanced_economy')
)


In [ ]:
def gen_dummy(columns:list):
    for cols in columns:
        yield (pl.when(pl.col(cols).is_null()).then(1).otherwise(0).name.suffix("_dummy"))

In [ ]:
columns = [
        'fertility_rate',
        'gdp_per_capita',
        'government_expenditure',
        'trade',
        'life_expectancy',
        'gdp_per_capita_growth',
        'inflation',
        'population_growth',
        'gross_capital_formation',
        'domestic_bank']

In [ ]:
df = df.with_columns(gen_dummy(columns=columns))
df2 = df

In [ ]:
country_df = df.group_by(["country"]).agg(
    cs.ends_with("_dummy").mean()
)
country_df = country_df.with_columns(total_pct=pl.sum_horizontal(cs.ends_with("_dummy")) * 100 / 12)
min_countries = country_df.filter(pl.col("total_pct") <= 5).select(pl.col("country")).to_series().to_list()
len(min_countries)

In [ ]:
df = dc.pull_wb()
data = pl.read_csv("data/external/countries.csv")
df = df.join(data, on="country", how="inner", validate="m:1")
df = df.select(
    pl.col(
        'year',
        'country',
        'fertility_rate',
        'gdp_per_capita',
        'government_expenditure',
        'trade',
        'life_expectancy',
        'gdp_per_capita_growth',
        'inflation',
        'population_growth',
        'gross_capital_formation',
        'domestic_bank',
        'advanced_economy')
)

In [ ]:
extra_contries = ['Iraq', 'Malta', 'Slovak Republic', 'Slovenia','Guinea', 'Mozambique','Namibia', 'Sierra Leone',"Solomon Islands",'Albania', 'Armenia', 'Austria', 'Azerbaijan', 'Belarus',
       'Belgium', 'Brunei Darussalam', 'Bulgaria', 'Cambodia', 'Cyprus',
       'Finland', 'France', 'Georgia', 'Germany', 'Greece', 'Ireland',
       'Italy', 'Kazakhstan', 'Kuwait', 'Kyrgyz Republic', 'Luxembourg',
       'Moldova', 'Mongolia', 'North Macedonia', 'Portugal', 'Spain',
       'Ukraine','Benin', 'Nicaragua', 'Oman', 'Romania', 'Uganda',
       'Aruba', 'Haiti', 'Hungary', 'Poland', 'Qatar']

In [ ]:
df = df2.filter(pl.col("country").is_in(min_countries))
df = df.filter(~pl.col("country").is_in(extra_contries))
df

In [ ]:
# flagged = df.with_columns(
#     all=pl.all_horizontal(
# 'fertility_rate',
#  'gdp_per_capita',
#  'government_expenditure',
#  'trade',
#  'life_expectancy',
#  'gdp_per_capita_growth',
#  'inflation',
#  'population_growth',
#  'gross_capital_formation',
#  'domestic_bank',
#  'advanced_economy')
# )
# remove = flagged.filter(pl.col("all").is_null()).select(pl.col("country")).to_series().unique().to_list()
# df = df.filter(pl.col("country").is_in(remove))
# df

In [ ]:
df.select(cs.ends_with("_dummy"))

In [ ]:
data = df.to_pandas()
data = data.sort_values(["year", "country"])
columns = [
    "fertility_rate",
    "gdp_per_capita",
    "government_expenditure",
    "trade",
    "life_expectancy",
    "gdp_per_capita_growth",
    "inflation",
    "population_growth",
    "gross_capital_formation",
    "domestic_bank",
    
]
for col in columns:
    data[col] = data.groupby("country")[col].transform(
        lambda group: group.interpolate(method="linear")
    )
data = data.sort_values(["country", "year"])
data = data[data["year"] >= 1990]
msno.matrix(data.sort_values(["country", "year"]))